In [3]:
import pandas as pd


In [4]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
# Q1. Downloading the data
# We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
# Read the data for January. How many columns are there?

len(df.columns) # 19

19

In [6]:
# Q2. Computing duration
# Now let's compute the duration variable. It should contain the duration of a ride in minutes.
# What's the standard deviation of the trips duration in January?


df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds() / 60
df['duration'].std() # 42.59

42.59435124195458

In [7]:
# Q3. Dropping outliers
# Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

# What fraction of the records left after you dropped the outliers?

df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

fraction = f'{(len(df_filtered) / len(df)) * 100}%'
fraction # 98.12%

'98.1220282212598%'

In [8]:
# Q4. One-hot encoding
# Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer. Get a feature matrix from it.
# What's the dimensionality of this matrix (number of columns)?

df_filtered['pickup_ids'] = df_filtered['PULocationID'].astype(str)
df_filtered['dropoff_ids'] = df_filtered['DOLocationID'].astype(str)

df_dict = df_filtered[['pickup_ids', 'dropoff_ids']].to_dict(orient='records')

from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
matrix = dv.fit_transform(df_dict)
matrix.shape  # 515

/var/folders/rn/qr7lg_zn25990k8pbb594l4c0000gn/T/ipykernel_68481/2009368799.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['pickup_ids'] = df_filtered['PULocationID'].astype(str)
/var/folders/rn/qr7lg_zn25990k8pbb594l4c0000gn/T/ipykernel_68481/2009368799.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['dropoff_ids'] = df_filtered['DOLocationID'].astype(str)


(3009173, 515)

In [9]:
# Q5. Training a model
# Now let's use the feature matrix from the previous step to train a model.
# Train a plain linear regression model with default parameters, where duration is the response variable
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?

X_train = matrix
y_train = df_filtered['duration'].values

from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error
import numpy as np

y_pred = lr.predict(X_train)
rmse = mean_squared_error(y_train, y_pred) # it does not accept the argument 'squared' so I use numpy to do that!
rmse = np.sqrt(rmse) 
rmse # 7.64

7.649261927686161

In [10]:
# Q6. Evaluating the model
# Now let's apply this model to the validation dataset (February 2023).
# What's the RMSE on validation?

df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_feb['duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb['duration'] = df_feb['duration'].dt.total_seconds() / 60

df_feb_filtered = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]

df_feb_filtered['pickup_ids'] = df_feb_filtered['PULocationID'].astype(str)
df_feb_filtered['dropoff_ids'] = df_feb_filtered['DOLocationID'].astype(str)

df_feb_dict = df_feb_filtered[['pickup_ids', 'dropoff_ids']].to_dict(orient='records')

X_feb = dv.transform(df_feb_dict)
y_val = df_feb_filtered['duration'].values
y_pred = lr.predict(X_feb)

rmse_feb = mean_squared_error(y_pred, y_val) # it does not accept the argument 'squared' so I use numpy to do that!
rmse_feb = np.sqrt(rmse_feb)
rmse_feb # 7.81

/var/folders/rn/qr7lg_zn25990k8pbb594l4c0000gn/T/ipykernel_68481/3518020768.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_filtered['pickup_ids'] = df_feb_filtered['PULocationID'].astype(str)
/var/folders/rn/qr7lg_zn25990k8pbb594l4c0000gn/T/ipykernel_68481/3518020768.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_filtered['dropoff_ids'] = df_feb_filtered['DOLocationID'].astype(str)


7.811817957524739

In [13]:
import pickle

with open('model.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [16]:
# Q1. Notebook
# We'll start with the same notebook we ended up with in homework 1. 
# We cleaned it a little bit and kept only the scoring part. You can find the initial notebook here.
# Run this notebook for the March 2023 data.
# What's the standard deviation of the predicted duration for this dataset?

import pickle
import pandas as pd


with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)


categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

df['pickup_ids'] = df['PULocationID'].astype(str)
df['dropoff_ids'] = df['DOLocationID'].astype(str)
dicts = df[['pickup_ids', 'dropoff_ids']].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
y_pred.std() # 6.24

6.247500973564958

In [18]:
# Q2. What's the size of the output file?

year = 2023
month = 3

# Create a new column 'ride_id'
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

# Create a new DataFrame to hold only the output columns
df_result = pd.DataFrame()

# Copy the ride_id to the new DataFrame
df_result['ride_id'] = df['ride_id']

# Add the predicted duration values to the new DataFrame
df_result['predicted_duration'] = y_pred

# Define the output filename based on the year and month
output_file = f'output_{year:04d}-{month:02d}.parquet'

# Save the result as a Parquet file using pyarrow
# No compression, and we don't include the index in the file
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

# 65.5MB ~ 66MB

In [19]:
# Q3. Now let's turn the notebook into a script.
# Which command you need to execute for that?

jupyter nbconvert --to script 04-deployment.ipynb


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2803037006.py, line 4)